In [1]:
import torch as t
from grid_world import STATES, ACTIONS, GridWorld
from agent import ParameterizedPolicyAgent, ValueIterationAgent
import pandas as pd
import plotly.express as px

C:\Users\saisa\AppData\Local\Temp\ipykernel_15052\1133365847.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
results = []

In [3]:
n_states, n_actions = len(STATES), len(ACTIONS)
SIGMA = 0.9
pp_agent = ParameterizedPolicyAgent(n_states, n_actions, SIGMA)

In [4]:
N_EPISODES = 50
N_TRIALS = 100

In [5]:
results.extend(pp_agent.hillSearch(N_EPISODES, N_TRIALS))

trial=0, avg return=-83.6
trial=1, avg return=-71.8
trial=4, avg return=-46.0
trial=6, avg return=-22.8
trial=10, avg return=-11.0
trial=18, avg return=-6.8
trial=23, avg return=0.2
trial=24, avg return=3.2
trial=25, avg return=9.0
trial=27, avg return=10.0


In [6]:
df = pd.DataFrame.from_records([r.to_dict() for r in results])
df.sample(n=10)

,Episode,Trial,Return,Sigma,NumEpisodesPerTrial,NumTrials
4055,5,82,10.0,0.9,50,100
1938,38,39,10.0,0.9,50,100
3085,35,62,10.0,0.9,50,100
3902,2,79,10.0,0.9,50,100
1530,30,31,10.0,0.9,50,100
4994,44,100,10.0,0.9,50,100
2181,31,44,10.0,0.9,50,100
1719,19,35,10.0,0.9,50,100
2355,5,48,10.0,0.9,50,100
3229,29,65,10.0,0.9,50,100


In [7]:
avg_df = df.groupby(by=["Sigma","Trial"], as_index=False).agg('mean')
std_df = df.groupby(by=["Sigma","Trial"], as_index=False).agg('std')
std_df.sample(n=5)
std_err = (std_df["Return"]/N_EPISODES).to_list()

In [8]:
fig = px.line(avg_df, x="Trial", y="Return", error_y=std_err, color="Sigma", title=f"N_EPISODES={N_EPISODES}")
fig.show()

In [9]:
results_val_iteration = []

In [56]:
n_states, n_actions = len(STATES), len(ACTIONS)
gw = GridWorld()
terminal_states, transistion_fn, reward_fn = [gw.goal_state], gw.getTransistionFn(), gw.getRewardFn()
GAMMA = 0.9
vi_agent = ValueIterationAgent(
    n_states,
    n_actions, 
    gamma=GAMMA, 
    theta=1e-6, 
    terminal_states=[gw.goal_state],
    transition_fn=transistion_fn,
    reward_fn=reward_fn)

112 tensor([ 4.4640e+00,  4.4197e+00,  3.9086e+00,  3.9346e+00,  4.4742e+00,
         5.0868e+00,  5.0627e+00,  4.4174e+00,  4.0459e+00,  4.6147e+00,
         5.8331e+00,  5.8047e+00, -8.1466e-06, -2.4475e-06,  6.6278e+00,
         6.7212e+00,  6.6091e+00,  7.5289e+00,  8.5165e+00,  9.6259e+00,
         8.3619e+00,  7.3846e+00,  8.4587e+00,  9.6916e+00,  0.0000e+00])


In [47]:
from agent import run_exp

In [48]:
N_TRIALS, N_EPISODES = 300, 50
avg_returns1, std_returns1 = run_exp(pp_agent, GridWorld(), N_TRIALS, N_EPISODES)
avg_returns2, std_returns2 = run_exp(vi_agent, GridWorld(), N_TRIALS, N_EPISODES)

In [49]:
std_err1 = [s/(N_EPISODES**0.5) for s in std_returns1]
std_err2 = [s/(N_EPISODES**0.5) for s in std_returns2]

In [63]:
fig = px.line(avg_returns2[:100], error_y=std_err2[:100], title=f"N_EPISODES={N_EPISODES}")
fig.show()

In [60]:
import plotly.graph_objects as go

x = list(range(N_TRIALS))
y1 = avg_returns1
y2 = avg_returns2

fig = go.Figure()
fig.update_layout(title=f'Avg return over {N_EPISODES} episodes per trial',
                   xaxis_title='Trial',
                   yaxis_title='Avg Return')
fig.update_layout(autosize=False, width=1700, height=500)
fig.add_trace(go.Scatter(x=x, y=y1, name=f"Parameterize policy agent (\sigma={SIGMA})"))
fig.add_trace(go.Scatter(x=x, y=y2, name=f"Value iteration agent (\gamma={GAMMA})"))
fig.show()